## A Spatio-Temporal Accessibility Analysis of Pharmacy Care in Vermont, USA
---

Extends and adapts studies *by* Kang, J. Y., A. Michels, F. Lyu, Shaohua Wang, N. Agbodo, V. L. Freeman, and Shaowen Wang. 2020. Rapidly measuring spatial accessibility of COVID-19 healthcare resources: a case study of Illinois, USA. International Journal of Health Geographics 19 (1):1–17. DOI:[10.1186/s12942-020-00229-x](https://ij-healthgeographics.biomedcentral.com/articles/10.1186/s12942-020-00229-x) AND Holler, J., Burt, D., Udoh, K., & Kedron, P. (2022). Reproduction and Reanalysis of Kang et al 2020 Spatial Accessibility of COVID-19 Health Care Resources. https://doi.org/10.17605/OSF.IO/N92V3



Authors: Sam Roubin, Joseph Holler, Peter Kedron

Reproduction Materials Available at: https://github.com/samroubin/VTPharmacy/tree/main

Created: `2024-01-14`
Revised: `2023-01-`

### Original Data
To perform the ESFCA method, three types of data are required, as follows: (1) road network, (2) population, and (3) pharmacy information. The road network can be obtained from the [OpenStreetMap Python Library, called OSMNX](https://github.com/gboeing/osmnx). The population data is available from the US Census Bureau on the [American Community Survey]. Lastly, hospital information available on our GitHub repository: 

### Modules
Import necessary libraries to run this model.
See `environment.yml` for the library versions used for this analysis.

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import re
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import folium
import itertools
import os
import time
import warnings
import IPython
import requests
from IPython.display import display, clear_output
from shapely.ops import nearest_points   #for hospital_setting function

warnings.filterwarnings("ignore")
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

In [15]:
# Import modules
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
from IPython.display import display, clear_output
from shapely.ops import nearest_points   #for hospital_setting function
import warnings
import os
from shapely.geometry import Point, LineString, Polygon
import sys

warnings.filterwarnings("ignore")
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
print("Python version:", sys.version)

pandas==2.2.0
geopandas==0.14.2
networkx==3.2.1
osmnx==1.8.1
Python version: 3.12.1 | packaged by conda-forge | (main, Dec 23 2023, 07:53:56) [MSC v.1937 64 bit (AMD64)]


## Check Directories


In [3]:
# Check working directory
os.getcwd()

'D:\\github\\samroubin\\VTPharmacy\\procedure\\code'

In [4]:
# Use to set work directory properly
if os.path.basename(os.getcwd()) == 'code':
    os.chdir('../../')
os.getcwd()

'D:\\github\\samroubin\\VTPharmacy'

## Load and Visualize Data

### Population by town for all states in study area (VT, NH, MA, NY)

In [ ]:
# Read in population data by town
population_df = gpd.read_file('./data/raw/public/population/tidycensus_population.gpkg')
population_df.head()


# Read in metropolitan / micropolitan classifications (NECTAS)
nectas_df =  gpd.read_file('./data/raw/public/population/nectas.csv')
nectas_df.head()


# Join NECTAS classifications to population data with subdivision and county FIPS
pop_df = pd.merge(population_df, nectas_df[['fips_subdivision', 'necta', 'fips_county', 'fips_state']], on=['fips_subdivision','fips_county', 'fips_state'], how = 'left')
pop_df.head()

# Save as geopackage into public/derived
pop_df.to_file('./data//derived/public/pop_data.gpkg', driver='GPKG')

pop_df.head()

In [ ]:
pop_df.plot()

### Load and Visualize Pharmacy Data

This data contains the hours of operations and the number of pharmacists and pharmacy technicians staffed at each pharmacy

In [ ]:
# Read in pharmacy data
pharmacies = gpd.read_file('./data/raw/public/pharmacy/pharmacies.gpkg')
# pd.set_option('display.max_rows', None)
pharmacies.head()

In [ ]:
# if pharmacy staffing file does not exist, download it from OSF 
if not os.path.exists("./data/raw/private/pharm_staffing.csv"):
    print("Loading pharmacy staffing levels", flush=True)
    url = ' https://osf.io/download/yvxqj/'
    r = requests.get(url, allow_redirects=True)
    open('./data/raw/private/pharm_staffing.csv', 'wb').write(r.content)
   
    
# Load pharmacy data from OSF project
pharm_staffing = pd.read_csv('./data/raw/private/pharm_staffing.csv')

pharm_staffing.head()

In [ ]:
# Join private staffing data to pharmacies dataset (locations, hours of operations, etc.) 
pharmacies_df = pd.merge(pharmacies, pharm_staffing, on='pharmid', how = 'left')
pharmacies_df.head()


### Fill in missing staffing data

### Generate and Plot Map of Pharmacies

In [ ]:
pharmacies_df.explore()

### Load the Road Network

If `Vermont_Network_Buffer.graphml` does not already exist, this cell will query the road network from OpenStreetMap.  

Each of the road network code blocks may take a few mintues to run.

In [5]:
%%time
# To create a new graph from OpenStreetMap, delete or rename the graph file (if it exists)
# AND set OSM to True
# This is more likely to work on a local computer than CyberGISX
OSM = True

# Define the place name for Vermont
place_name_vermont = 'Vermont, USA'

roads_path = "./data/raw/private/osm_roads.graphml"

# if buffered street network is not saved, and OSM is preferred, generate a new graph from OpenStreetMap and save it
if not os.path.exists(roads_path) and OSM:
    print("Loading buffered Vermont road network from OpenStreetMap. Please wait... runtime may exceed 9min...", flush=True)
    G = ox.graph_from_place('Vermont', network_type='drive', buffer_dist=64373.8) 
    print("Saving road network to", roads_path, " Please wait...", flush=True)
    ox.save_graphml(G, roads_path)
    print("Data saved.")
    
# otherwise, if buffered street network is not saved, download graph from the OSF project
elif not os.path.exists(roads_path):
    print("Downloading buffered Vermont road network from OSF...", flush=True)
    url = 'https://osf.io/download/n2q73/'  
    r = requests.get(url, allow_redirects=True)
    print("Saving road network to", roads_path, " Please wait...", flush=True)
    open(roads_path, 'wb').write(r.content)
    
# load the saved network graph
if os.path.exists(roads_path):
    print("Loading road network from", roads_path, "Please wait...", flush=True)
    G = ox.load_graphml(roads_path) 
    print("Data loaded.") 
else:
    print("Error: could not load the road network from file.")

Loading buffered Vermont road network from ./data/raw/private/osm_roads.graphml Please wait...
Data loaded.
CPU times: total: 41.7 s
Wall time: 42.1 s


### Plot the Road Network

In [ ]:
%%time
ox.plot_graph(G, node_size = 1, bgcolor = 'white', node_color = 'black', edge_color = "#333333", node_alpha = 0.5, edge_linewidth = 0.5)

#### Check speed limits and highway types

Display all the unique speed limit values and count how many network edges (road segments) have each value.
We will compare this to our cleaned network later.

In [6]:
%%time
# Turn network edges into a geodataframe
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Count frequency of each speed value
speed_values = edges['maxspeed'].value_counts()

# Ouput number of edges and frequences of speed values
print(str(len(edges)) + " edges in graph")
print(speed_values.to_string())

577399 edges in graph
maxspeed
30 mph                                              48848
40                                                  20564
30                                                  18812
50                                                  15082
25 mph                                              14227
35 mph                                              10340
40 mph                                               9657
55 mph                                               6035
45 mph                                               5790
50 mph                                               3702
70                                                   2089
20 mph                                               1343
90                                                   1246
100                                                  1005
65 mph                                                722
[40 mph, 30 mph]                                      470
15 mph                                   

Display all the unique highway types, which are used to impute the speed limits for each category of highway.

In [ ]:
# view all highway types
print(edges['highway'].value_counts())

The OSMNx algorithm to impute missing speed limit data assumes that units are kilometers per hour unless the units are specified.
Therefore, search for network edges containing speed limit data without units within the United States, where speed limits are defined in miles per hour.

In [ ]:
# find edges with speed values
unitless = edges[edges.maxspeed.notna()]

# find edges with speed values containing only a number without units
unitless = unitless[unitless.maxspeed.str.isnumeric().fillna(False)]

# explore unitless edges south of Canadian border (45 degrees latitude)
unitless[unitless.bounds['maxy']<45].explore()

# 503 *More than 503 with increased buffer* road segments contain speed data without units, and these are almost exclusively in Canada.
# It hardly seems worthwhile to fix these few segments. 

Only *503* (different now) edges in the network contain speed limit information without units, and very few of these are located within the United States.

### Process the road network

Impute speed limits with `add_edge_speeds` function. 

In [7]:
%%time
ox.speed.add_edge_speeds(G)

CPU times: total: 9.95 s
Wall time: 10 s


Calculate travel time with the `add_edge_travel_times` function.

In [8]:
%%time
ox.speed.add_edge_travel_times(G)

CPU times: total: 10.9 s
Wall time: 11 s


Add a point geomerty to each node in the graph, to facilitate constructing catchment area polygons later on.

In [11]:
# Create point geometries for each node in the graph, to make constructing catchment area polygons easier
for node, data in G.nodes(data=True):
    data['geometry']=Point(data['x'], data['y'])

#### Check speed limits
Display all the unique speed limit values and count how many network edges (road segments) have each value.

In [12]:
%%time
# Turn network edges into a geodataframe
nodes,edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

# Count frequency of each speed value
speed_values = edges['speed_kph'].value_counts()

# Ouput number of edges and frequences of speed values
print(str(len(edges)) + " edges in graph")
print(speed_values.to_string())

577399 edges in graph
speed_kph
42.0     308436
48.3      48848
51.6      35227
50.2      32018
40.0      20736
30.0      18812
54.0      17268
50.0      15096
40.2      14227
63.9      14004
56.3      10340
64.4       9657
88.5       6035
72.4       5790
80.5       3702
70.0       2180
75.6       1729
67.6       1398
32.2       1343
90.0       1252
100.0      1006
80.0        812
104.6       722
60.0        720
55.2        514
56.0        507
68.0        483
24.1        467
72.0        419
45.6        391
35.0        370
52.0        333
62.1        320
76.0        286
64.0        281
47.7        278
16.1        161
44.0        145
48.0        137
33.5        117
112.7        79
20.0         76
57.0         72
45.0         71
84.0         50
75.0         49
25.0         45
67.0         39
85.0         38
65.0         36
61.0         27
15.0         26
36.0         21
96.3         20
108.0        19
96.0         17
96.6         16
27.4         14
10.0         14
69.0         12
55.0    

In [14]:
edges['osmid'] = edges['osmid'].astype('str') 
edges['highway'] = edges['highway'].astype('str')
edges[["osmid", "geometry", "highway", "oneway", "speed_kph"]].to_file('./data/scratch/edges.shp')
nodes.to_file('./data/scratch/nodes.gpkg')

Speeds have been imputed and converted to kilometers per hour.

In [ ]:
# calculate a color scheme for edges based on speed
ec = ox.plot.get_edge_colors_by_attr(G, attr="speed_kph", cmap='viridis')

# plot edge speeds
fig, ax = ox.plot_graph(G, node_size=0, edge_color=ec, bgcolor="white")

# note: the aesthetics of this could be improved!

In [16]:
nodes

,y,x,street_count,geometry,highway,ref
osmid,,,,,,
61538824,42.556628,-71.993666,3,POINT (-71.99367 42.55663),NaN,NaN
61540048,42.566053,-71.992658,3,POINT (-71.99266 42.56605),NaN,NaN
61540293,42.553004,-71.973578,3,POINT (-71.97358 42.55300),NaN,NaN
61541585,42.550698,-71.971501,3,POINT (-71.97150 42.55070),NaN,NaN
61542977,42.566653,-72.004361,3,POINT (-72.00436 42.56665),NaN,NaN
...,...,...,...,...,...,...
11541693369,45.536855,-73.644975,3,POINT (-73.64497 45.53686),NaN,NaN
11541693370,45.538545,-73.643906,3,POINT (-73.64391 45.53854),NaN,NaN
11541693371,45.537971,-73.644245,3,POINT (-73.64424 45.53797),NaN,NaN


The graph of speed limits looks accurate, with local roads at low speeds and state and federal highways at higher speeds.

## "Helper" Functions

These functions are called when the model is run. 

### pharmacy_setting

Finds the nearest network node for each pharmacy.

Args:

* pharmacies: GeoDataFrame of pharmacies
* G: OSMNX network

Returns:

* GeoDataFrame of pharmacies with info on nearest network node

In [ ]:
nodes_osmid = gpd.GeoDataFrame(nodes["geometry"]).reset_index()
nodes_osmid.head()

In [ ]:
pharmacies_osm = gpd.sjoin_nearest(pharmacies_df, nodes_osmid, distance_col="distances")

#rename column from osmid to nearest_osm, so that it works with other code
pharmacies_osm = pharmacies_osm.rename(columns={"osmid": "nearest_osm"})
pharmacies_osm.head()


### djikstra_cca_polygons

Function written by Joe Holler + Derrick Burt. A more efficient way to calculate distance-weighted catchment areas for each hospital in Holler et al. (2022). We use this method to calculate catchment areas for each pharmacy.  First, create a dictionary (with a node and its corresponding drive time from the hospital) of all nodes within a 30 minute drive time (using networkx single_cource_dijkstra_path_length function). From here, two more dictionaries are constructed by querying the original one. From these dictionaries, single part convex hulls are created for each drive time interval and appended into a single list (one list with 3 polygon geometries). Within the list, the polygons are differenced from each other to produce three catchment areas.

Args:
* G: cleaned network graph *with node point geometries attached*
* nearest_osm: A unique nearest node ID calculated for a single pharmacy
* distances: 3 distances (in drive time) to calculate catchment areas from

Returns:
* A list of 3 differenced (not-overlapping) catchment area polygons (10 min poly, 20 min poly, 30 min poly)

In [ ]:
def dijkstra_cca_polygons(G, nearest_osm, distances):
    
## Distance_unit is given in seconds ##

    ## CREATE DICTIONARIES ##
    # create dictionary of nearest nodes
    nearest_nodes_30 = nx.single_source_dijkstra_path_length(G, nearest_osm, distances[2], "travel_time") # creating the largest graph from which 10 and 20 minute drive times can be extracted from

    # extract values within 20 and 10 (respectively) minutes drive times
    nearest_nodes_20 = dict()
    nearest_nodes_10 = dict()
    for key, value in nearest_nodes_30.items():
        if value <= distances[1]:
            nearest_nodes_20[key] = value
        if value <= distances[0]:
            nearest_nodes_10[key] = value

    ## CREATE POLYGONS FOR 3 DISTANCE CATEGORIES (10 min, 20 min, 30 min) ##

    # 30 MIN
    # If the graph already has a geometry attribute with point data,
    # this line will create a GeoPandas GeoDataFrame from the nearest_nodes_30 dictionary
    points_30 = gpd.GeoDataFrame(gpd.GeoSeries(nx.get_node_attributes(G.subgraph(nearest_nodes_30), 'geometry')))

    # This line converts the nearest_nodes_30 dictionary into a Pandas data frame and joins it to points
    # left_index=True and right_index=True are options for merge() to join on the index values
    points_30 = points_30.merge(pd.Series(nearest_nodes_30).to_frame(), left_index=True, right_index=True)

    # Re-name the columns and set the geodataframe geometry to the geometry column
    points_30 = points_30.rename(columns={'0_x':'geometry','0_y':'z'}).set_geometry('geometry')

    # Create a convex hull polygon from the points
    polygon_30 = gpd.GeoDataFrame(gpd.GeoSeries(points_30.unary_union.convex_hull))
    polygon_30 = polygon_30.rename(columns={0:'geometry'}).set_geometry('geometry')

    # 20 MIN # 1200 seconds!
    # Select nodes less than or equal to 20
    points_20 = points_30.query("z <= 1200")

    # Create a convex hull polygon from the points
    polygon_20 = gpd.GeoDataFrame(gpd.GeoSeries(points_20.unary_union.convex_hull))
    polygon_20 = polygon_20.rename(columns={0:'geometry'}).set_geometry('geometry')

    # 10 MIN # 600 seconds!
    # Select nodes less than or equal to 10
    points_10 = points_30.query("z <= 600")

    # Create a convex hull polygon from the points
    polygon_10 = gpd.GeoDataFrame(gpd.GeoSeries(points_10.unary_union.convex_hull))
    polygon_10 = polygon_10.rename(columns={0:'geometry'}).set_geometry('geometry')

    # Create empty list and append polygons
    polygons = []

    # Append
    polygons.append(polygon_10)
    polygons.append(polygon_20)
    polygons.append(polygon_30)

    # Clip the overlapping distance ploygons (create two donuts + hole)
    for i in reversed(range(1, len(distances))):
        polygons[i] = gpd.overlay(polygons[i], polygons[i-1], how="difference")

    return polygons

### pharmacy_measure_acc (adjusted to incorporate dijkstra_cca_polygons)

Measures the effect of a single pharmacy on the surrounding area. (Uses `dijkstra_cca_polygons`)

Args:

* pharmacies_df: Geopandas dataframe with information on a pharmacy
* distances: Distances in time to calculate accessibility 
* weights: how to weight the different travel distances

Returns:

* Tuple containing:
    * Int (\_thread\_id)
    * GeoDataFrame of catchment areas with key stats

In [ ]:
def pharmacy_measure_acc (pharmacies_df, distances, weights):
    
    # Create polygons
    polygons = dijkstra_cca_polygons(G, pharmacies['nearest_osm'], distances)
    
    # update polygons with time, and ICU beds. Set CRS to 4326, then convert to 6589
    for i in range(len(distances)):
        polygons[i]['time']=distances[i]
        polygons[i].crs = { 'init' : 'epsg:4326'}
        polygons[i] = polygons[i].to_crs({'init':'epsg:6589'})
    
    return([ polygon.copy(deep=True) for polygon in polygons ]) 

### measure_acc_par

Parallel implementation of accessibility measurement.

Args:

* pharmacies: Geodataframe of pharmacies
* network: OSMNX street network
* distances: list of distances to calculate catchments for
* weights: list of floats to apply to different catchments

Returns:

* Geodataframe of catchments with accessibility statistics calculated

In [ ]:
def measure_acc_par (pharmacies_df, network, distances, weights):
    
    # initialize catchment list, 3 empty geodataframes
    catchments = []
    for distance in distances:
        catchments.append(gpd.GeoDataFrame())
        
    # pool = mp.Pool(processes = num_proc)
    
    # makes a list of all pharmacy info.  len = 192
    # looks like this, except with all info, and for all 192 hospitals
    # [[2, Methodist Hospital of Chicago, Chicago], [4, Advocate Christ Medical Center, Oak Lawn]]
    pharmacy_list = [ pharmacies_df.iloc[i] for i in range(len(pharmacies_df)) ]
    
    print("Calculating", len(pharmacy_list), "pharmacy catchments...\ncompleted number:", end=" ")
    
    # call pharmacy_acc_unpacker
    # returns a tuple containing the thread ID and a list of copied polygons
    #results = pool.map(hospital_acc_unpacker, zip(range(len(hospital_list)), hospital_list, itertools.repeat(pop_data), itertools.repeat(distances), itertools.repeat(weights)))
    
    results = []  
    for i in range(len(pharmacy_list)): #do from 1 to 66
        result = pharmacy_measure_acc(i, pharmacy_list[i], distances, weights)
        results.append(result)

    # pool.close()
    
    # sort and extract the results
    results.sort()
    results = [ r[1] for r in results ]
    
    # combine catchment results into the respective GeoDataFrames in the catchments list
    for i in range(len(results)):
        for j in range(len(distances)):
            catchments[j] = catchments[j].append(results[i][j], sort=False)
            
    return catchments

In [ ]:
distances = [600, 1200, 1800] # Distances in travel time (seconds!)
weights = [1.0, 0.68, 0.22] 

# initialize catchment list, 3 empty geodataframes
catchments = []
for distance in distances:
    catchments.append(gpd.GeoDataFrame())
    
    
results = gpd.GeoDataFrame(columns = ["geometry","pharmid","weight"], crs = "EPSG:4326", geometry = "geometry")
for ind in pharmacies_osm.index:
    print("Working on pharmacy", pharmacies_osm['pharmid'][ind])
     ## CREATE DICTIONARIES ##
    # create dictionary of nearest nodes
    nearest_nodes_30 = nx.single_source_dijkstra_path_length(G, pharmacies_osm['nearest_osm'][ind], distances[2], "travel_time") # creating the largest graph from which 10 and 20 minute drive times can be extracted from

    # extract values within 20 and 10 (respectively) minutes drive times
    nearest_nodes_20 = dict()
    nearest_nodes_10 = dict()
    for key, value in nearest_nodes_30.items():
        if value <= distances[1]:
            nearest_nodes_20[key] = value
        if value <= distances[0]:
            nearest_nodes_10[key] = value

    ## CREATE POLYGONS FOR 3 DISTANCE CATEGORIES (10 min, 20 min, 30 min) ##

    # 30 MIN
    # If the graph already has a geometry attribute with point data,
    # this line will create a GeoPandas GeoDataFrame from the nearest_nodes_30 dictionary
    points_30 = gpd.GeoDataFrame(gpd.GeoSeries(nx.get_node_attributes(G.subgraph(nearest_nodes_30), 'geometry')))

    # This line converts the nearest_nodes_30 dictionary into a Pandas data frame and joins it to points
    # left_index=True and right_index=True are options for merge() to join on the index values
    points_30 = points_30.merge(pd.Series(nearest_nodes_30).to_frame(), left_index=True, right_index=True)

    # Re-name the columns and set the geodataframe geometry to the geometry column
    points_30 = points_30.rename(columns={'0_x':'geometry','0_y':'z'}).set_geometry('geometry')

    # Create a convex hull polygon from the points
    polygon_30 = gpd.GeoDataFrame(gpd.GeoSeries(points_30.unary_union.convex_hull))
    polygon_30 = polygon_30.rename(columns={0:'geometry'}).set_geometry('geometry')
    polygon_30["weight"] = weights[2]

    # 20 MIN # 1200 seconds!
    # Select nodes less than or equal to 20
    points_20 = points_30.query("z <= 1200")

    # Create a convex hull polygon from the points
    polygon_20 = gpd.GeoDataFrame(gpd.GeoSeries(points_20.unary_union.convex_hull))
    polygon_20 = polygon_20.rename(columns={0:'geometry'}).set_geometry('geometry')
    polygon_20["weight"] = weights[1]

    # 10 MIN # 600 seconds!
    # Select nodes less than or equal to 10
    points_10 = points_30.query("z <= 600")

    # Create a convex hull polygon from the points
    polygon_10 = gpd.GeoDataFrame(gpd.GeoSeries(points_10.unary_union.convex_hull))
    polygon_10 = polygon_10.rename(columns={0:'geometry'}).set_geometry('geometry')
    polygon_10["weight"] = weights[0]
    
    # Clip the overlapping distance ploygons (create two donuts + hole)
    polygon_30_hole = gpd.overlay(polygon_30, polygon_20, how ="difference")
    polygon_20_hole = gpd.overlay(polygon_20, polygon_10, how ="difference")
    
    # Create dataframe combining polygon_10, polygon_20, polygon_30
    polygons = pd.concat([polygon_10, polygon_20_hole, polygon_30_hole])
    polygons.set_crs(crs="EPSG:4326", inplace = True)
    # polygons = pd.concat([polygon_10, polygon_20, polygon_30])
    # polygons.set_crs(crs="EPSG:4326", inplace = True)
    
    
    polygons_gdf = polygons
    polygons_gdf["pharmid"] = pharmacies_osm['pharmid'][ind]
    #polygons_gdf["weight"] = weights

    results = pd.concat([results, polygons_gdf])

In [ ]:
results.explore()

In [ ]:
# Find for which pharmacies there are issues with geometry and thus their polygon catchments
summary_table = results['pharmid'].value_counts().reset_index()
display(summary_table) #VT14, MA35, VT74, VT99 only have on geometry row

In [ ]:
# Remove pharmacies without correct geometries
#results_clean = results.drop(results[results['pharmid'] == ["VT14","MA35","VT74","VT99"]].index)
results_clean = results[~results['pharmid'].isin(["VT14","MA35","VT74","VT99"])]
results_clean.head()

In [ ]:
# Change CRS to match 
results_clean.to_crs("EPSG:6589", inplace = True)
pop_df.to_crs("EPSG:6589", inplace = True)

In [ ]:
# Calculate town areas
pop_df['town_area'] = pop_df.geometry.area
#display(pop_df)

#results_clean['s_area'] = results_clean.geometry.area

# Run the overlay to find intersection of fragments
fragments = gpd.overlay(pop_df, results_clean, how = 'intersection')

# Calculate fragment areas
fragments['frag_area'] = fragments.geometry.area

# Calculate area ratios
fragments['area_ratio']= fragments['frag_area'] / fragments['town_area']

# Calculate fragment value by multiplying area_ratio by distance weight
fragments['value'] = fragments['weight'] * fragments['area_ratio']

fragments.explore()

In [ ]:
# group_by pharmid and GEOID, then sum values to summarize the fragment values by pharmid
sum_results = fragments.groupby(by = ['pharmid','GEOID']).sum()
sum_results.head()


In [ ]:
# Calculate population served in each fragment
sum_results['pop_fragment'] = sum_results['value'] * sum_results['total_pop']
sum_results.head()

# Convert pharmacies dataset to EPSG 6589 to match other datasets
pharmacies_df.to_crs("EPSG:6589", inplace = True)

# Calculate service to population ratio for each fragment
accessibility_df = sum_results.merge(pharmacies_df, on='pharmid')
accessibility_df['week_staff'] = accessibility_df['week_pharm'] + 0.5 * accessibility_df['week_tech']
accessibility_df['serv_pop_week'] = accessibility_df['pop_fragment'] / accessibility_df['week_staff']
accessibility_df.head()

# Multiply by weight (again) and summarize by town 



In [ ]:
pharmacies_df.to_crs("EPSG:6589", inplace = True)
pharmacies_df.crs

### overlapping_function

Calculates how all catchment areas overlap with and affect the accessibility of each grid in our grid file.

Args:

* vt_towns: GeoDataFrame of our grid (VT towns)
* catchments: GeoDataFrame of our catchments
* service_type: the kind of care being provided (ICU beds vs. ventilators) XX
* weights: the weight to apply to each service type
* num\_proc: the number of processors

Returns:

* Geodataframe - grid\_file with calculated stats

#### Make "grid file" of VT towns

In [ ]:
def overlapping_function (vt_towns, catchments, service_type, weights, num_proc = 4):

    ## Area Weighted Reaggregation
        # switch to True for area-weighted overlay
    weighted = True

    # if the value to be calculated is already in the hegaxon grid, delete it
    # otherwise, the field name gets a suffix _1 in the overlay step
    if resource in list(vt_towns.columns.values):
        vt_towns = vt_towns.drop(resource, axis = 1)

    # calculate hexagon 'target' areas
    vt_towns['area'] = vt_towns.area

    # Intersection overlay of pharmacy catchments and towns grid
    print("Intersecting pharmacy catchments with town grid...")
    fragments = gpd.overlay(vt_towns, geocatchments, how='intersection')

    # Calculate percent coverage of the hexagon by the hospital catchment as
    # fragment area / target(hexagon) area
    fragments['percent'] = fragments.area / fragments['area']

    # if using weighted aggregation... 
    if weighted:
        print("Calculating area-weighted value...")
        # multiply the service/population ratio by the distance weight and the percent coverage
        fragments['value'] = fragments[resource] * fragments['weight'] * fragments['percent']

    # if using the 50% coverage rule for unweighted aggregation...
    # else:
    #     print("Calculating value for hexagons with >=50% overlap...")
    #     # filter for only the fragments with > 50% coverage by hospital catchment
    #     fragments = fragments[fragments['percent']>=0.5]
    #     # multiply the service/population ration by the distance weight
    #     fragments['value'] = fragments[resource] * fragments['weight']

    # select just the hexagon id and value from the fragments,
    # group the fragments by the (hexagon) id,
    # and sum the values
    print("Summarizing results by town id...")
    sum_results = fragments[['id', 'value']].groupby(by = ['id']).sum()

    # join the results to the hexagon grid_file based on hexagon id
    print("Joining results to hexagons...")
    results = pd.merge(vt_towns, sum_results, how="left", on = "id")

    # rename value column name to the resource name 
    return(results.rename(columns = {'value' : resource}))

### normalization

Normalizes our result (Geodataframe).

In [ ]:
def normalization (result, resource):
    result[resource]=(result[resource]-min(result[resource]))/(max(result[resource])-min(result[resource]))
    return result

### file_import

Imports all files we need to run our code and pulls the Illinois network from OSMNX if it is not present (will take a while). 

**NOTE:** even if we calculate accessibility for just Chicago, we want to use the Illinois network (or at least we should not use the Chicago network) because using the Chicago network will result in hospitals near but outside of Chicago having an infinite distance (unreachable because roads do not extend past Chicago).

Args:

* pop_type: population type, either "pop" for general population or "covid" for COVID-19 cases
* region: the region to use for our hospital and grid file ("Chicago" or "Illinois")

Returns:

* G: OSMNX network
* hospitals: Geodataframe of hospitals
* grid_file: Geodataframe of grids
* pop_data: Geodataframe of population

In [ ]:
def output_map(output_grid, base_map, hospitals, resource):
    ax=output_grid.plot(column=resource, cmap='PuBuGn',figsize=(18,12), legend=True, zorder=1)
    # Next two lines set bounds for our x- and y-axes because it looks like there's a weird 
    # Point at the bottom left of the map that's messing up our frame (Maja)
    ax.set_xlim([314000, 370000])
    ax.set_ylim([540000, 616000])
    base_map.plot(ax=ax, facecolor="none", edgecolor='gray', lw=0.1)
    hospitals.plot(ax=ax, markersize=10, zorder=1, c='blue')

## Run the model

Below you can customize the input of the model:

* Processor - the number of processors to use
* Population - the population to calculate the measure for
* Resource - the hospital resource of interest
* Hospital - all hospitals or subset to check code

### Process population data

In [ ]:
''' 
To simplify the reanalysis, in variables I will hardcode the use of 
    4 processors
    Population: Population at Risk
    Resource: ICU Beds
    Hospital: All hospitals
'''

resource = "hospital_icu_beds"
num_proc = 4
pop_type = "pop"

## Create centroids for atrisk population at the census tract level
pop_data = pop_centroid(atrisk_data, pop_type)    
    
distances = [600, 1200, 1800] # Distances in travel time (seconds!)
weights = [1.0, 0.68, 0.22] # Weights where weights[0] is applied to distances[0]

In [ ]:
pop_data

### Process hospital data

In [ ]:
hospitals

In [ ]:
#Finds the nearest network node for each hospital
hospitals = hospital_setting(hospitals, nodes)

In [ ]:
hospitals


### Visualize catchment areas for hospital #4

In [ ]:
# Create point geometries for entire graph

# which hospital to visualize? 
fighosp = 4

# Create catchment for hospital 4
poly = dijkstra_cca_polygons(G, hospitals['nearest_osm'][fighosp], distances)

# Reproject polygons
for i in range(len(poly)):
    poly[i].crs = { 'init' : 'epsg:4326'}
    poly[i] = poly[i].to_crs({'init':'epsg:32616'})

# Reproject hospitals 
hospital_subset = hospitals.iloc[[fighosp]].to_crs(epsg=32616)

fig, ax = plt.subplots(figsize=(12,8))

min_10 = poly[0].plot(ax=ax, color="royalblue", label="10 min drive")
min_20 = poly[1].plot(ax=ax, color="cornflowerblue", label="20 min drive")
min_30 = poly[2].plot(ax=ax, color="lightsteelblue", label="30 min drive")

hospital_subset.plot(ax=ax, color="red", legend=True, label = "hospital")

# Add legend
ax.legend()

### Calculate hospital catchment areas

In [ ]:
%%time

catchments = measure_acc_par(hospitals, pop_data, G, distances, weights, num_proc)

### Calculate accessibility

### Post-process the catchments (for area weighted reaggregation)

In [ ]:
# add weight field to each catchment polygon
for i in range(len(weights)):
    catchments[i]['weight'] = weights[i]
# combine the three sets of catchment polygons into one geodataframe
geocatchments = pd.concat([catchments[0], catchments[1], catchments[2]])
geocatchments

### Area Weighted Reaagregation

In [ ]:
%%time
result = overlapping_function(grid_file, catchments, resource, weights, num_proc)

In [ ]:
%%time
result = normalization (result, resource)

## Results & Discussion

Extensive cleaning of unneccesary variables and lines of code that were never called.

### Making code more efficient and easier to read with GeoPandas

1. Made the __pop_centroid__ function much faster - preciosly took 3:30 to run, now less than a second. Instead of creating an empty GDF and iterating over all of the population geometries, adding data to this new GDF, I just used the native GeoPandas centroid method, replacing the population geometries with centroids, and then dropping other unnecessary columns from atrisk_data. 

2. Rewrote the __hospital_setting__ function to find each hospital's nearest node using GeoPandas nearest join method. What took 1:20 to run now runs in less than a second. I also cleaned the GDF so that it matched what we were working with before. 

### Removed parallel processing from two functions. 

1. __overlapping_function__
2. __measure_acc_par__

### Theoretical Changes to the methodology

Area weighted reaggregation - 
assigned speeds to the road network using osnmx. 

### Simplifying Code for future students

My greatest contribution to this replication has been the simplification of code and adding documentation to functions. This has made the code much easier for future students to read through and understand, and has not sacrificed processing times. I also made a visual workflow, visualizing the replication study from start to finish, including all data and functions used to manipulate them. 

Simplifications include:

I removed the dropdown menu that allows you to choose between population groups and hospital data. The benefits of this dropdown options were minimal, and it just made the code more confusing to follow and modify. In the form of a dropdown selection, it prevents the study from being one script, and introduces potential error as groups try to replicate eachother, if they are not clear about which choices they made with their mouse in the dropdown. 

I was able to delete the function __overlap_calc__, after implementing its function into __overlapping_function__ which was implements the area weighted reaggregation.

I removed a code block that filtered rows where the "hospital_icu_beds" value is infinity, which did not do anything. 



### Accessibility Map

In [ ]:
%%time
hospitals = hospitals.to_crs({'init': 'epsg:26971'})
result = result.to_crs({'init': 'epsg:26971'})
output_map(result, pop_data, hospitals, resource)

Classified Accessibility Outputs

### Conclusion

to be written.

### References

Luo, W., & Qi, Y. (2009). An enhanced two-step floating catchment area (E2SFCA) method for measuring spatial accessibility to primary care physicians. Health & place, 15(4), 1100-1107.